# **Cancel Culture - Classification Modeling Notebook**

---

**Post-Cleaning Modeling Notebook**

---

# -- > 🛑 **FIX**: Add cmts re: post-cleaning, modeling

---

>

---

# **Imports**

---

> I will import several packages and modules to perform my modeling steps. I use different parts of Scikit-Learn's numerous packages for the majority of my modeling. In addition to SKLearn, I tested models from XGBoost as well as Imblearn towards the end of the modeling process.
>
> I include the usual packages as well: pandas, numpy, matplotlib, and seaborn. Additionally, I have several personal functions that I use during the modeling process.

---

In [ ]:
## Jupyter Notebook setting to reload functions when called
%load_ext autoreload
%autoreload 2

In [ ]:
## Data Handling
import pandas as pd
import numpy as np

## Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

## Personal functions
from bmc_functions import classification as clf

## SKLearn and Modeling Tools
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn import set_config
set_config(display='diagram')

from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:
## Settings
%matplotlib inline
sns.set_context("paper", font_scale=1.25)
pd.set_option('display.max_columns', 150)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 50)

# **Reading the DataFrames**

---

> In my prior EDA notebook, I reviewed, cleaned, and performed some pre-processing steps to prepare my data separately before modeling. I saved the data as a .pickle file to preserve the datatypes; now I will re-read the data for modeling purposes.

---

In [ ]:
data = pd.read_pickle('./data/data_prepped.pickle',
                           compression = 'gzip')
data.head(5)

# **Train/Test Split**

In [ ]:
## Identifying target
target= 'is_canceled'

In [ ]:
## Dropping target and "reservation_status" (nearly identical indicator)

X = data.drop(columns = [target, 'reservation_status']).copy()
y = data[target].copy()

In [ ]:
## Checking for missing values
print(f'Missing values for X:\n {X.isna().sum()[X.isna().sum() >0]}\n')
print(f'Missing values for y: {y.isna().sum()}')

In [ ]:
## Splitting - stratify to maintain class balance b/t X_train/_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, 
                                                    random_state=42, 
                                                    stratify=y)

In [ ]:
## Saving memory by deleting unused X, y
del X, y

In [ ]:
## Specifying numeric columns for preprocessing
num_cols = X_train.select_dtypes('number').columns.to_list()
num_cols

In [ ]:
## Specifying numeric columns for preprocessing
cat_cols = X_train.select_dtypes(include='object').columns.to_list()
cat_cols

# **Prepping the Pipeline**

---

> Pipeline to streamline modeling steps:
* Preprocessing: OHE, scaling, outliers via ƒ-XF?
* Modeling: RFC, BRFC
* GSCV: include as part of pipeline
* Get results:
    * Feature importances - **SHAP**

---

In [ ]:
## Creating ColumnTransformer and sub-transformers for imputation and encoding

### --- Creating column pipelines --- ###

cat_pipe = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore',
                                                 sparse=False))])

num_pipe = Pipeline(steps=[('scaler', StandardScaler())])

## Instantiating the ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[('num', num_pipe, num_cols),
                  ('cat', cat_pipe, cat_cols)
                  ])

preprocessor

In [ ]:
## Fitting feature preprocessor
preprocessor.fit(X_train)

## Getting feature names from OHE
ohe_cat_names = preprocessor.named_transformers_['cat'].named_steps['ohe'].get_feature_names(cat_cols)

## Generating list for column index
final_cols = [*num_cols, *ohe_cat_names]

final_cols

In [ ]:
## Transform the data via the ColumnTransformer preprocessor

X_train_tf = preprocessor.transform(X_train)
X_train_tf_df = pd.DataFrame(X_train_tf, columns=final_cols, index=X_train.index)

X_test_tf = preprocessor.transform(X_test)
X_test_tf_df = pd.DataFrame(X_test_tf, columns=final_cols, index=X_test.index)

display(X_train_tf_df.head(5),X_test_tf_df.head(5))

---

> Due to class imbalance, will attempt to use "class_weight = balanced" to correct.

---

# **Speeding-Up Scikit-Learn**

---

Due to the size of my dataset, the modeling process took a fair amount of time, especially when testing different model types. To improve my models' runtime, I use a package called "**Intel(R) Extension for Scikit-learn*.**"

This package operates in the background to increase the computational efficiency of certain Scikit-Learn models, including Logistic Regression and Random Forest Classifier models. The package does not affect the model results.

---

In [ ]:
## Speeding up SKLearn via Intel(R) Extension for Scikit-learn*
from sklearnex import patch_sklearn
patch_sklearn()

# **Baseline Model**

In [ ]:
## Creating baseline classifier model

base = DummyClassifier(strategy='stratified', random_state = 42)

base.fit(X_train_tf_df, y_train)

clf.evaluate_classification(base,X_train = X_train_tf_df, y_train = y_train,
                           X_test = X_test_tf_df, y_test = y_test, 
                           metric = 'balanced accuracy')

# **Logistic Regression Model**

---

**Results:**

> Training balanced accuracy score: 0.82
> 
> Testing balanced accuracy score: 0.82
> 
> * *The scores are the same size.*
>
> Training data log loss: 0.37
>
> Testing data log loss: 0.37

---

**Interpretation**

> 

---

In [ ]:
## LogReg Model

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter = 600,C = .1,solver = 'lbfgs',
                            n_jobs=-1,random_state = 42)

logreg.fit(X_train_tf_df, y_train)

In [ ]:
clf.evaluate_classification(logreg, X_train = X_train_tf_df, y_train = y_train,
                           X_test = X_test_tf_df, y_test = y_test,
                          metric = 'balanced recall')

## Gathering Results

---

> I feel that this regression model is showing strong results, so I will collect the results for my features and generate a visualization of the results.

---

In [ ]:
## Collecting coefficients for each feature as a Series
lr_coefs = pd.Series(logreg.coef_.flatten(), index=X_train_tf_df.columns)
lr_coefs.sort_values(ascending=False, inplace=True)
lr_coefs

In [ ]:
## Converting top/bottom 5 values into a Series
log_odds = pd.concat([lr_coefs.head(5), lr_coefs.tail(5)])
log_odds

In [ ]:
## Formatting index labels to become visualization labels
new_labels_list = [i.replace('_', ' ').title() for i in list(log_odds.index)]
new_labels_list

In [ ]:
## Creating a dictionary to replace the old lables with the new ones
new_labels_dict = { k:v for (k,v) in zip(log_odds.index, new_labels_list)}
new_labels_dict

In [ ]:
## Renaming Series index
log_odds = log_odds.rename(new_labels_dict)
log_odds.sort_values(inplace=True)

log_odds

In [ ]:
## Visualizing log-odds

fig, ax = plt.subplots(figsize=(7,4))

ax = log_odds.plot(kind='barh', ax=ax)
ax.axvline(linestyle = '-', c='k')
ax.set_xlabel('Log-Odds')
ax.set_ylabel('Feature Name')
ax.set_title('Top and Bottom Five Features')
ax.set_facecolor('0.9')
fig.set_facecolor('0.975')
# plt.savefig('./img/log_odds.png',transparent=False, bbox_inches='tight',
#            dpi=100)
plt.show()
plt.close()

---

***May the (Log-)Odds be Ever in Your Favor***

> Based on the logistic regression model coefficients, I see that reservations are **most likely to cancel** if they:
* Require non-refundable deposits (possibly 3rd-party booking sites like )
* Are booked by agents 17 or 240
* Have previous cancellations
* Guests are from Portugal ([PRT is the three letter ISO 3166-1 code for Portugal](https://en.wikipedia.org/wiki/Portugal#:~:text=ISO%203166%20code,PT))
>
> Alternatively, reservations are **least likely to cancel** if they:
* Do NOT require a deposit
* Require parking spaces
* Reserve room type "A"
* Are assigned to room type "I"
* Are booked by agent 152

***Oddities in the Results***

> **Non-Refundable Deposit Requirement**
* May be associated with 3rd party travel groups like Priceline/Expedia/etc.
    * Often require pre-payment to the booking group/agent to confirm booking
>
> **Country of Origin: Portugal**
* Could these hotels be located in Portugal and have a larger percentage of domestic travelers?
>
> **Room Assignment, Required Car Parking Spaces**
* These features may be generated post-stay and may not be available prior to arrival
* Parking spot requirements may be specified prior to arrival, but not as likely in my personal experience.

---

## **GridSearchCV - Logistic Regression**

---

**Results:**

> *Results unavailable - did not run model due to time required*

---

**Interpretation**

> 

---

In [ ]:
# lr_params = {
#  'C': [.001, .01, .1, 1],
#     'penalty':['l1', 'l2'],#, 'elasticnet', 'none'],
#     'solver':['newton-cg', 'liblinear', 'sag', 'saga', 'lbfgs'],
#     'max_iter':[100, 300, 500]}

# gscv = GridSearchCV(LogisticRegression(class_weight='balanced'), lr_params,
#                     scoring = 'balanced_accuracy', cv=3, n_jobs = -1)
# gscv

# gscv.fit(X_train_tf_df, y_train)

In [ ]:
# clf.evaluate_classification(gscv.best_estimator_, X_train = X_train_tf_df, y_train = y_train,
#                            X_test = X_test_tf_df, y_test = y_test,
#                           metric = 'balanced accuracy')

In [ ]:
# logreg_params = gscv.best_params_

# logreg_params

In [ ]:
# gscv.best_estimator_

# **Random Forest Model**

---

**Results:**

> Training balanced recall score: 0.99
> 
> Testing balanced recall score: 0.88
>
> * *The training score is larger by 0.11 points.*
>
> Training data log loss: 0.08
>
> Testing data log loss: 0.27

---

**Interpretation**

> 

---

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(class_weight = 'balanced', n_jobs=-1,
                             random_state=42)

rfc.fit(X_train_tf_df, y_train)

In [ ]:
clf.evaluate_classification(rfc, X_train = X_train_tf_df, y_train = y_train,
                           X_test = X_test_tf_df, y_test = y_test,
                          metric = 'balanced recall')

In [ ]:
clf.plot_importances(rfc, X_train_tf_df)

## **GSCV - RandomForest**

---

**Scores**

> Training balanced accuracy score: 0.99
> 
> Testing balanced accuracy score: 0.88
> 
> * *The training score is larger by 0.11 points.*
>
> Training data log loss: 0.10
>
> Testing data log loss: 0.26

---

**Best Parameters**

> 'criterion': 'entropy'
> 
> 'max_depth': None
> 
> 'min_samples_leaf': 2
> 
> 'min_samples_split': 2

**Interpretation**

> 

---

In [ ]:
# rfc_params = {
#     'criterion': ['gini', 'entropy'],
#     'max_depth': [25,50, None],
#     'min_samples_split': [2,3,4],
#     'min_samples_leaf': [1,2]
# }

In [ ]:
# rfgs = GridSearchCV(RandomForestClassifier(class_weight = 'balanced',
#                                            random_state = 42, n_jobs=-1),
#                     rfc_params,scoring = 'balanced_accuracy',
#                     cv=3,verbose = 4)

# rfgs.fit(X_train_tf_df, y_train)

In [ ]:
# rfgs.best_params_

In [ ]:
# rfgs.best_score_

In [ ]:
# rfc_new = rfgs.best_estimator_

In [ ]:
# clf.evaluate_classification(rfc_new, X_train_tf_df, y_train, X_test_tf_df, 
#                            y_test, 'balanced recall')

# **ExtraTreesClassifier**

---

**Results:**

> Training balanced recall score: 1.0
> 
> Testing balanced recall score: 0.87
> 
> * 
>
> Training data log loss: 0.01
>
> Testing data log loss: 0.33

---

**Interpretation**

> 

---

In [ ]:
# from sklearn.ensemble import ExtraTreesClassifier

# etc = ExtraTreesClassifier(class_weight = 'balanced', n_jobs=-1, random_state=42)

# etc.fit(X_train_tf_df, y_train)

In [ ]:
# clf.evaluate_classification(etc, X_train = X_train_tf_df, y_train = y_train,
#                            X_test = X_test_tf_df, y_test = y_test,
#                           metric = 'balanced recall')

# **Balanced Random Forest Classifier**

---

**MODEL: BalancedRandomForestClassifier**

**Scores**

> Training balanced accuracy score: 0.97
> 
> Testing balanced accuracy score: 0.89
> 
> * *The training score is larger by 0.8 points.*
>
> Training data log loss: 0.17
>
> Testing data log loss: 0.30

---

**Best Parameters**

> 

**Interpretation**

> 

---

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

balanced_forest = BalancedRandomForestClassifier(n_jobs = -1, random_state=42)

balanced_forest.fit(X_train_tf_df, y_train)

In [ ]:
clf.evaluate_classification(balanced_forest, X_train = X_train_tf_df, y_train = y_train,
                           X_test = X_test_tf_df, y_test = y_test,
                          metric = 'balanced recall')

## **GSCV: Balanced Random Forest Classifier**

---

**MODEL:** GridSearchCV - BalancedBaggingClassifier

**Scores**

> Training balanced accuracy score: 0.96
> 
> Testing balanced accuracy score: 0.89
> 
> * *The training score is larger by 0.07 points.*
>
> Training data log loss: 0.20
>
> Testing data log loss: 0.30

---

**Best Parameters**

> * 'criterion': 'entropy'
>
> * 'max_features': None
>
> * 'min_samples_leaf': 1
>
> * 'min_samples_split': 2

**Interpretation**

> 

---

In [ ]:
brfc_params = {'criterion': ['gini', 'entropy'],
               'min_samples_split': [2, 3, 4],
               'min_samples_leaf': [1,2,3]
              }

In [ ]:
brfc = GridSearchCV(BalancedRandomForestClassifier(n_jobs=-1,random_state=42), 
                    brfc_params,scoring = 'balanced_accuracy',
                    cv=3,verbose = 4)

brfc.fit(X_train_tf_df, y_train)

In [ ]:
clf.evaluate_classification(brfc, X_train_tf_df, y_train, X_test_tf_df, 
                           y_test, 'balanced precision')

In [ ]:
brfc_params = brfc.best_params_

brfc_params

In [ ]:
brfc.best_score_

In [ ]:
brfc_params = {'criterion': ['entropy'],
               'min_samples_split': [2],
               'min_samples_leaf': [1],
               'class_weight': ['balanced', 'balanced_subsample']
              }

In [ ]:
brfc = GridSearchCV(BalancedRandomForestClassifier(n_jobs=-1,random_state=42), 
                    brfc_params,scoring = 'balanced_accuracy',
                    cv=3,verbose = 4)

brfc.fit(X_train_tf_df, y_train)

In [ ]:
clf.evaluate_classification(brfc, X_train_tf_df, y_train, X_test_tf_df, 
                           y_test, 'balanced precision')

# **Interpretation**

---

**Odd Features**

> Now that I completed my modeling steps, I will review the results of each model and determine my final recommendations.
>
> My main models are a standard logistic regression and a Balanced Random Forest classifier ("BRFC"). **Each model provides a different way of identifying which features are most impactful: logistic regressions provide "log-odds" and a Balanced Random Forest Classifier produces "feature importances."** Both will require some processing for easy interpretation.

**Feature Importances and SHAP**

> As I mention above, tree-based models, including my BRFC-model, return "feature importances" instead of the coefficients associated with linear/logistic regressions. These values are useful to show the impact of a given feature on the decision-making steps of the tree model. 
>
>However, these feature importances suffer from one key weakness: *they do not indicate if a feature increases or decreases the likelihood of a reservation canceling (my target feature).*
>
> I will utilize a visualization package called **SHAP** to produce "Shapely values" for each feature. These values indicate each feature's marginal contribution to the model - answering the question, "*How well does the model perform with this feature than without?*" 

**Seeing is Believing**

>Using tools within the package, I will focus on two visualizations:
> * The `summary_plot`: visualizing each feature's Shapely value and the feature's values from low-high (relative to each feature).
>
>
> * The `force_plot`: an in-depth look at the forces impacting any given reservation record.
>
> More information about SHAP:
* [SHAP Documentation](https://shap.readthedocs.io/en/latest/?badge=latest)
* [SHAP Repository](https://github.com/slundberg/shap)

---

# **SHAP**

---

**Purpose**

> 

**Process**

> 

**Performance**

> 

---

# 🛑 Fix: Annotate Code and Update

In [ ]:
import shap 
print(shap.__version__)
shap.initjs()

In [ ]:
## May need to gather sub-sample of data, ex:

X_shap = shap.sample(X_test_tf_df,nsamples=200)
explainer = shap.TreeExplainer(brfc.predict,X_shap)
explainer

In [ ]:
# ## Initialize an explainer with the model
# explainer = shap.TreeExplainer(rf)

## Calculaate shap values for test data
shap_values = explainer.shap_values(X_test_tf_df,y_test)
len(shap_values)

In [ ]:
shap_values[1].shape, X_test_tf_df.shape

In [ ]:
## Bar

shap.summary_plot(shap_values[1], X_test_tf_df,plot_type='bar',max_display=40)

In [ ]:
## Better plot

shap.summary_plot(shap_values[1],X_test_tf_df,max_display=40)

## **Force Plot**

In [ ]:
target_lookup = {0:'Check-Out',1:'Canceled'}
target_lookup[1]

In [ ]:
row = np.random.choice(range(len(X_test_tf_df)))
print(f"- Row #: {row}")
print(f"Class = {target_lookup[y_test.iloc[row]]}")
X_test_tf_df.iloc[row].round(2)

In [ ]:
## Individual forceplot
shap.force_plot(explainer.expected_value[1], shap_values[1][row],X_test_tf_df.iloc[row])       

In [ ]:
## Overall Forceplot
shap.force_plot(explainer.expected_value[1], shap_values[1],X_test_tf_df)       

## Dependence Plot

In [ ]:
shap.dependence_plot('Lead Time',shap_values[1],X_test_tf_df)

# MVP Notes

* CLF results - feature importances
* feature importances - visualize via SHAP


# Reviewing Results

---

> After testing several models, I found that MODELNAMEHERE produced the most accurate results.
>
> TOP 5 STRONGEST INDICATORS - Canceled Reservations:
> * 
> * 
> * 
>
> TOP 5 STRONGEST INDICATORS - Actualized Reservations:
> * 
> * 
> * 

---

# Recommendations

---

> Operationally, these results give us data-supported insights into our future guests and their needs. Once deployed, hotels would be able to use this model to forecast potential occupancy and staffing/supply needs. 
>
> Additionally, Operations teams would be able to determine how many and which guests would be the most likely to cancel their reservations. This information is very useful during periods of high-occupancy, particularly when trying to determine which guests to relocate in case of an oversold hotel.

---

# Future Work

---

> Time series modeling - forecasting based off of daily average probabilities (for a given # of arrivals, what is the forecast of %/# CXL?

> TSM - vector autoregressive forecasting using features to predict # cxl

> *Major stretch goal/future work:* determining likelihood of cancellations at given thresholds - e.g. 0-3, 4-7, 7-14, etc. days out
* What would be feature importances/coefficients at each threshold?
* Could I group the reservations based on their lead time despite different years?

---